In [75]:
# Importing all the libraries needed
import numpy as np
import pandas as pd
import scipy 
import matplotlib as mplt
import os
from pydub import AudioSegment
from python_speech_features import mfcc, logfbank, ssc
import scipy.io.wavfile as wav
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [76]:
# Some variable and path initializations
os.chdir("/home/alex/Desktop/Bits,Coding and Extra/Bits_pilani_projects_and_competitions/Apogee_Project/apogee_amgc/genres1")
song_no = 0
df = pd.DataFrame(columns=['mfcc_feat','fbank_feat','ssc','classification'],index=range(0,1000))
copy_df = pd.DataFrame(columns=['mfcc_feat','fbank_feat','ssc','classification'],index=range(0,1000))

In [88]:
# Getting all the genres
l = []
for dirpaths,dirnames,filenames in os.walk(os.getcwd()):
    l.append(dirnames)

In [78]:
# Extracting Features of the songs

for i in l[0]:
    #for genre in genres1
    for x in range(100):
        # for song in batch of songs
        
        # Extracting the features
        (rate,sig) = wav.read(i+'/'+i+'.000'+"%02d"%x+'.wav')
        mfcc_feat = mfcc(sig,rate,nfft=551)
        fbank_feat = logfbank(sig,rate,nfft=551)
        sig_temp = np.reshape(sig, (sig.shape[0],1))
        ssc_var = ssc(sig_temp,rate,nfft=551)
        
        #Adding features to the pandas dataframe
        df.iloc[song_no][0] = mfcc_feat[0:2992,:]
        df.iloc[song_no][1] = fbank_feat[0:2992,:]
        df.iloc[song_no][2] = ssc_var[0:2992,:]
        df.iloc[song_no][3] = i
        
        # Incrementing the song number
        song_no+=1
        
    

In [ ]:
# Doing the same operation twice, because of bug with deepcopy function in pandas 

song_no = 0

# Extracting Features of the songs

for i in l[0]:
    #for genre in genres1
    for x in range(100):
        # for song in batch of songs
        
        # Extracting the features
        (rate,sig) = wav.read(i+'/'+i+'.000'+"%02d"%x+'.wav')
        mfcc_feat = mfcc(sig,rate,nfft=551)
        fbank_feat = logfbank(sig,rate,nfft=551)
        sig_temp = np.reshape(sig, (sig.shape[0],1))
        ssc_var = ssc(sig_temp,rate,nfft=551)
        
        #Adding features to the pandas dataframe
        copy_df.iloc[song_no][0] = mfcc_feat[0:2992,:]
        copy_df.iloc[song_no][1] = fbank_feat[0:2992,:]
        copy_df.iloc[song_no][2] = ssc_var[0:2992,:]
        copy_df.iloc[song_no][3] = i
        
        # Incrementing the song number
        song_no+=1
        
    

In [ ]:
## Standardizing the features of the songs ##

# Splitting into train and test
train, test = train_test_split(copy_df, test_size=0.2)

# Normalizing according to each feature in each frame over all songs
for frame in range(2992):
    for feature in range(65):
        temp = []  # Empty list
        for song in range(800):
            if feature < 13: # part of mfcc features
                temp.append( (train.iloc[song,0])[frame,feature] )
            elif feature >=13 and feature < 26 : # part of fbeat features
                temp.append( (train.iloc[song,1])[frame,feature - 13] )
            else: # part of ssc features
                temp.append( (train.iloc[song,2])[frame,feature - 39] )

        temp_arr = np.array(temp) # converting it to a rank 1 array
        temp_arr = temp_arr.reshape(800,1) # converting into normal column vector
        para = StandardScaler()
        stand = para.fit_transform(temp_arr) 
    
        # updating the values in train
        for song2 in range(800):
            if feature < 13: # part of mfcc features
                (train.iloc[song2,0])[frame,feature] = stand[song2]
            elif feature >=13 and feature < 26 : # part of fbeat features
                (train.iloc[song2,1])[frame,feature - 13] = stand[song2]
            else: # part of ssc features
                (train.iloc[song2,2])[frame,feature - 39] = stand[song2]

        # transforming the test set
        temp3 = []
        for song3 in range(200):
            if feature < 13: # part of mfcc features
                temp3.append( (test.iloc[song3,0])[frame,feature] )
            elif feature >=13 and feature < 26 : # part of fbeat features
                temp3.append( (test.iloc[song3,1])[frame,feature - 13] )
            else: # part of ssc features
                temp3.append( (test.iloc[song3,2])[frame,feature - 39] )
        temp3_arr = np.array(temp3) # converting it to a rank 1 array
        temp3_arr = temp3_arr.reshape(200,1)  # converting into normal column vector
        stand3 = para.transform(temp3_arr) # normalizing one feature of the same frame over all songs

        # updating the values in test
        for song4 in range(200):
            if feature < 13: # part of mfcc features
                (test.iloc[song4,0])[frame,feature] = stand3[song4]
            elif feature >=13 and feature < 26 : # part of fbeat features
                (test.iloc[song4,1])[frame,feature - 13] = stand3[song4]
            else: # part of ssc features
                (test.iloc[song4,2])[frame,feature - 39] = stand3[song4]
                

In [ ]:
print((copy_df.iloc[0,0])[0,0])

(copy_df.iloc[0,0])[0,0] = 0.2

print((copy_df.iloc[0,0])[0,0])
print((df.iloc[0,0])[0,0])

(copy_df.iloc[0,0])[0,0] = 21.2633158908
print((copy_df.iloc[0,0])[0,0])
print((df.iloc[0,0])[0,0])

In [ ]:
## Applying PCA ##

features = ['mfcc_feat','fbank_feat','ssc']

#Separating out the features, fitting and tranforming the train and test data
train_values = train.loc[:,features].values # returns an array from the dataframe

for i in range(3):
    t = StandardScaler().fit_transform(train_values[0])

train_normalized = StandardScaler().fit_transform(train_values)

# Transforming with the same scale for test values
test_values = train.loc[:,features].values
test_normalized = StandardScaler().transform(test_values)

#Separating out the target
train_classification = train.iloc[:,['classification']].values
test_classification = test.iloc[:,['classification']].values

# Number of components is 8
pca = PCA( n_components = 8) 

principal_components_train = pca.fit_transform(train_normalized)
principal_components_test = pca.fit_transform(test_normalized)

pca_train_temp = pd.Dataframe(data = principal_components_train, columns = ['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8']) 
pca_test_temp = pd.Dataframe(data = principal_components_test,columns = ['pca1','pca2','pca3','pca4','pca5','pca6','pca7','pca8'])

pca_train_df = pd.concat( [pca_train_temp,train_classification], axis = 1 )
pca_test_df = pd.concat( [pca_test_temp,test_classification], axis = 1 )

print(pca_train_df)
print(pca_test_df)
